# Phan 2: phan tich cau truc quan

## Tai mo hinh va phan tich cau

In [12]:
import spacy
from spacy import displacy
# Tải mô hình tiếng Anh đã cài đặt
# Sử dụng en_core_web_md vì nó chứa các vector từ và cây cú pháp đầy đủ
nlp = spacy.load("en_core_web_md")
# Câu ví dụ
text = "The quick brown fox jumps over the lazy dog."
# Phân tích câu với pipeline của spaCy
doc = nlp(text)

## Truc quan hoa cay phu thuoc

In [9]:
# Tùy chọn để hiển thị trong trình duyệt
# options = {"compact": True, "color": "blue", "font": "Source Sans Pro"}
# Khởi chạy server tại http://127.0.0.1:5000
# Bạn có thể truy cập địa chỉ này trên trình duyệt để xem cây phụ thuộc
# Nhấn Ctrl+C trong terminal để dừng server
displacy.serve(doc, style="dep")


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [02/Dec/2025 14:53:16] "GET / HTTP/1.1" 200 7538
127.0.0.1 - - [02/Dec/2025 14:53:16] "GET /favicon.ico HTTP/1.1" 200 7538


Shutting down server on port 5000.


In [13]:
for token in doc:
    print(token.text, token.pos_, token.dep_)

The DET det
quick ADJ amod
brown ADJ amod
fox NOUN nsubj
jumps VERB ROOT
over ADP prep
the DET det
lazy ADJ amod
dog NOUN pobj
. PUNCT punct


# Phần 3: Truy cập các thành phần trong cây phụ thuộc

In [17]:
# Lấy một câu khác để phân tích
text = "Apple is looking at buying U.K. startup for $1 billion"
doc = nlp(text)
# In ra thông tin của từng token
print(f"{'TEXT':<12} | {'DEP':<10} | {'HEAD TEXT':<12} | {'HEAD POS':<8} | {'CHILDREN'}")
print("-" * 70)

for token in doc:
    # Trích xuất các thuộc tính
    children = [child.text for child in token.children]
    print(f"{token.text:<12} | {token.dep_:<10} | {token.head.text:<12} | {token.head.pos_:<8} | {children}")

TEXT         | DEP        | HEAD TEXT    | HEAD POS | CHILDREN
----------------------------------------------------------------------
Apple        | nsubj      | looking      | VERB     | []
is           | aux        | looking      | VERB     | []
looking      | ROOT       | looking      | VERB     | ['Apple', 'is', 'at']
at           | prep       | looking      | VERB     | ['buying']
buying       | pcomp      | at           | ADP      | ['startup']
U.K.         | compound   | startup      | NOUN     | []
startup      | dobj       | buying       | VERB     | ['U.K.', 'for']
for          | prep       | startup      | NOUN     | ['billion']
$            | quantmod   | billion      | NUM      | []
1            | compound   | billion      | NUM      | []
billion      | pobj       | for          | ADP      | ['$', '1']


# Phần 4: Duyệt cây phụ thuộc để trích xuất thông tin

## Bài toán: Tìm chủ ngữ và tân ngữ của một động từ

In [5]:
text = "The cat chased the mouse and the dog watched them."
doc = nlp(text)
for token in doc:
    # Chỉ tìm các động từ
    if token.pos_ == "VERB":
        verb = token.text
        subject = ""
        obj = ""
        # Tìm chủ ngữ (nsubj) và tân ngữ (dobj) trong các con của động từ
        for child in token.children:
            if child.dep_ == "nsubj":
                subject = child.text
            if child.dep_ == "dobj":
                obj = child.text
            if subject and obj:
                print(f"Found Triplet: ({subject}, {verb}, {obj})")

Found Triplet: (cat, chased, mouse)
Found Triplet: (cat, chased, mouse)
Found Triplet: (cat, chased, mouse)
Found Triplet: (dog, watched, them)
Found Triplet: (dog, watched, them)


## Bài toán: Tìm các tính từ bổ nghĩa cho một danh từ

In [6]:
text = "The big, fluffy white cat is sleeping on the warm mat."
doc = nlp(text)
for token in doc:
    # Chỉ tìm các danh từ
    if token.pos_ == "NOUN":
        adjectives = []
        # Tìm các tính từ bổ nghĩa (amod) trong các con của danh từ
        for child in token.children:
            if child.dep_ == "amod":
                adjectives.append(child.text)
            if adjectives:
                print(f"Danh từ '{token.text}' được bổ nghĩa bởi các tính từ: {adjectives}")

Danh từ 'cat' được bổ nghĩa bởi các tính từ: ['big']
Danh từ 'cat' được bổ nghĩa bởi các tính từ: ['big']
Danh từ 'cat' được bổ nghĩa bởi các tính từ: ['big', 'fluffy']
Danh từ 'cat' được bổ nghĩa bởi các tính từ: ['big', 'fluffy', 'white']
Danh từ 'mat' được bổ nghĩa bởi các tính từ: ['warm']


## Bai tap tu luyen

In [32]:
# bai 1
def find_main_verb(sentence: str):
    doc = nlp(sentence)
    for token in doc:
        if token.dep_ == "ROOT":
            return token
    return None

# bai 2
def get_noun_chunks(sentence: str):
    doc = nlp(sentence)
    result = []
    for token in doc:
        if token.pos_ == "NOUN":
            noun_chunk = []
            for child in token.children:
                if child.dep_ in ["det", "amod", "compound"]:
                    noun_chunk.append(child.text)
            noun_chunk.append(token.text)
            noun_chunk = " ".join(noun_chunk)
            result.append(str(noun_chunk))
    return result


# bai 3
def get_path_to_root(token):
    result = [token.text]
    while True:
        head = token.head
        result.append(head.text)
        if head.dep_ == "ROOT":
            return "->".join(result)
        token = head


# test
## test bai1
sentence = "The quick brown fox jumps over the lazy dog."
root_token = find_main_verb(sentence)
print(f"Root token of sentence: {sentence} - {root_token}\n")


## test bai 2
noun_chunks = get_noun_chunks(sentence)
print(f"Noun chunks from sentence: {sentence}: \n{noun_chunks}\n")


## test bai 3
doc = nlp(sentence)
for token in doc:
    root_path = get_path_to_root(token)
    print(f"Path to the root of '{token}' from '{sentence}': \n{root_path}\n")
    print('----')


Root token of sentence: The quick brown fox jumps over the lazy dog. - jumps

Noun chunks from sentence: The quick brown fox jumps over the lazy dog.: 
['The quick brown fox', 'the lazy dog']

Path to the root of 'The' from 'The quick brown fox jumps over the lazy dog.': 
The->fox->jumps

----
Path to the root of 'quick' from 'The quick brown fox jumps over the lazy dog.': 
quick->fox->jumps

----
Path to the root of 'brown' from 'The quick brown fox jumps over the lazy dog.': 
brown->fox->jumps

----
Path to the root of 'fox' from 'The quick brown fox jumps over the lazy dog.': 
fox->jumps

----
Path to the root of 'jumps' from 'The quick brown fox jumps over the lazy dog.': 
jumps->jumps

----
Path to the root of 'over' from 'The quick brown fox jumps over the lazy dog.': 
over->jumps

----
Path to the root of 'the' from 'The quick brown fox jumps over the lazy dog.': 
the->dog->over->jumps

----
Path to the root of 'lazy' from 'The quick brown fox jumps over the lazy dog.': 
lazy->d